In [5]:
%reset
from osgeo import gdal
import numpy as np
 
def data_fusion(ortho_NIR, ortho_rgb, ortho_mspec, outputpath):

    #open the images  
    g = gdal.Open(ortho_NIR)
    c = gdal.Open(ortho_mspec)
    x = gdal.Open(ortho_rgb)
    d = gdal.Open(dem)
    
    # reporject the image to the same
    #dsReprj_mspec= gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_mspec_5.tif", c, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    dsReprj_nir = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_nir_5.tif", g, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    dsReprj_dem = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_dem_5.tif", d, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    dsReprj_rgb = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_rgb_5.tif", x, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    
    stack = []
    
    # retrieve the Bands you want and append them to the stack
    # RGB
    rgb_red = dsReprj_rgb.GetRasterBand(1).ReadAsArray().astype(np.float32)
    rgb_green = dsReprj_rgb.GetRasterBand(2).ReadAsArray().astype(np.float32)
    rgb_blue =  dsReprj_rgb.GetRasterBand(3).ReadAsArray().astype(np.float32)
    stack.append(rgb_red)
    stack.append(rgb_green)
    stack.append(rgb_blue)
    
    # Red Edge
    #mspec_rededge = dsReprj_mspec.GetRasterBand(3).ReadAsArray().astype(np.float32) 
    #stack.append(mspec_rededge)
    
    # NIR and NDVI (calculated from NIR and red)
    b3 = dsReprj_nir.GetRasterBand(1).ReadAsArray().astype(np.float32)
    b4 = dsReprj_nir.GetRasterBand(4).ReadAsArray().astype(np.float32)
    nir_ndvi = (b4 - b3)/(b4 + b3)
    nir_nir = dsReprj_nir.GetRasterBand(4).ReadAsArray().astype(np.float32)
    stack.append(nir_nir)
    stack.append(nir_ndvi)
    
    # DEM
    dem_chanel = dsReprj_dem.GetRasterBand(1).ReadAsArray().astype(np.float32)
    stack.append(dem_chanel)
    
    print("Channel count", len(stack))
    stack_data = stack
    #stack_data = np.asarray(stack, dtype=object)
    #stack_data = np.array(stack, dtype=object)

    # check the size of the shape
    yvalue = 0
    xvalue = 0
    for i in stack_data:
        if yvalue < i.shape[0]:
            yvalue = i.shape[0]

        if xvalue < i.shape[1]:
            xvalue = i.shape[1]
    
    print("Maximum Value: ", xvalue, "Minimum Value: ", yvalue)
   
    # save the new created stack as a an image with n-Layers
    driver = gdal.GetDriverByName ( "GTiff" )
    DataSet = driver.Create(outputpath, xvalue, yvalue, len(stack_data), gdal.GDT_Float32)
    DataSet.SetGeoTransform(x.GetGeoTransform())
    DataSet.SetProjection(x.GetProjection())
    for i, image in enumerate(stack_data, 1):
        DataSet.GetRasterBand(i).WriteArray(image)
        DataSet.GetRasterBand(i).SetNoDataValue(-9999.0)
    DataSet.FlushCache()

    print("Finished     .tiff file created")
    return None

Nothing done.


In [6]:
#ortho_NIR = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/nir_4.tif"
#ortho_rgb = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/rgb_4.tif"
#ortho_mspec = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/mspec_4.tif"
#dem = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/DEM_gross.tif"
#outputpath = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/verschiedene Stacks/Stack_rgb_dem_ndvi_nir.tif"

#ortho_NIR   = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction/Wetzwil_pred_nir3.tif"
#ortho_rgb   = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction/Wetzwil_pred_rgb3.tif"
#ortho_mspec = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction/Wetzwil_pred_mspec3.tif"
#dem         = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction/Wetzwil_pred_dem3.tif"
#outputpath  = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction/Stack_rgb_6Channels.tif"
##outputpath2 = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction/Stack_2.tif"

ortho_NIR   = "D:/cbr/VP2/VP2_SoilMapping/Processed/Gempelen/Planet_rgb_nir.tif"
ortho_rgb   = "D:/cbr/VP2/VP2_SoilMapping/Processed/Gempelen/rgb.tif"
ortho_mspec = "D:/cbr/VP2/VP2_SoilMapping/Processed/Gempelen/Planet_rgb_nir.tif"
dem         = "D:/cbr/VP2/VP2_SoilMapping/Processed/Gempelen/dem.tif"
outputpath  = "D:/cbr/VP2/VP2_SoilMapping/Processed/Gempelen/Stack_6Channels.tif"

data_fusion(ortho_NIR, ortho_rgb, ortho_mspec, outputpath)

C:\Users\ST1154~1\AppData\Local\Temp/ipykernel_14412/1048978701.py:35: RuntimeWarning: invalid value encountered in true_divide
  nir_ndvi = (b4 - b3)/(b4 + b3)


Channel count 6
Maximum Value:  20398 Minimum Value:  10792
Finished     .tiff file created
